In [2]:
import sys
import numpy as np
from PySide6.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QPushButton, QFileDialog
from PySide6.QtCore import Qt
import open3d as o3d
import open3d.visualization.gui as gui
import open3d.visualization.rendering as rendering

class CalibrationApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Calibration Visualization Tool")
        self.setGeometry(100, 100, 1200, 800)
        
        # Main widget and layout
        main_widget = QWidget()
        self.setCentralWidget(main_widget)
        layout = QVBoxLayout(main_widget)
        
        # Create Open3D visualization widget
        self.vis_widget = gui.Application.instance.create_window("3D Viewer", 800, 600)
        self.scene = gui.SceneWidget()
        self.vis_widget.add_child(self.scene)
        
        # Add controls
        self.load_button = QPushButton("Load Calibration Data")
        self.load_button.clicked.connect(self.load_calibration_data)
        layout.addWidget(self.load_button)
        
        # Initialize scene
        self.init_scene()
        
    def init_scene(self):
        # Set up basic scene with coordinate frame
        coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0)
        self.scene.scene.add_geometry("coord_frame", coord_frame)
        
        # Set default camera view
        bounds = self.scene.scene.bounding_box
        self.scene.setup_camera(60, bounds, bounds.get_center())
        
    def add_camera_visualization(self, position, orientation, color=[1, 0, 0]):
        """Add a camera visualization at the specified pose"""
        # Create camera frustum geometry
        camera_size = 0.2
        camera_mesh = o3d.geometry.TriangleMesh.create_arrow(
            cylinder_radius=0.02,
            cone_radius=0.04,
            cylinder_height=camera_size*0.8,
            cone_height=camera_size*0.2
        )
        
        # Transform camera to correct pose
        camera_mesh.paint_uniform_color(color)
        camera_mesh.transform(np.vstack([
            np.hstack([orientation, position.reshape(-1, 1)]),
            [0, 0, 0, 1]
        ]))
        
        return camera_mesh
        
    def add_point_cloud(self, points, colors=None):
        """Add a point cloud to the visualization"""
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
        
        if colors is not None:
            pcd.colors = o3d.utility.Vector3dVector(colors)
        else:
            # Default color if none provided
            pcd.paint_uniform_color([0, 1, 0])
            
        return pcd
        
    def load_calibration_data(self):
        """Load calibration data from file"""
        filename, _ = QFileDialog.getOpenFileName(
            self,
            "Load Calibration Data",
            "",
            "Calibration Files (*.json *.yaml);;All Files (*)"
        )
        
        if filename:
            # TODO: Implement your calibration data loading logic here
            # This is a placeholder example:
            
            # Example: Add some cameras
            cam1_pos = np.array([0, 0, 0])
            cam1_rot = np.eye(3)
            cam1 = self.add_camera_visualization(cam1_pos, cam1_rot, [1, 0, 0])
            self.scene.scene.add_geometry("camera1", cam1)
            
            cam2_pos = np.array([1, 0, 0])
            cam2_rot = np.eye(3)
            cam2 = self.add_camera_visualization(cam2_pos, cam2_rot, [0, 0, 1])
            self.scene.scene.add_geometry("camera2", cam2)
            
            # Example: Add some points
            points = np.random.rand(100, 3)
            point_cloud = self.add_point_cloud(points)
            self.scene.scene.add_geometry("points", point_cloud)
            
            # Update view
            bounds = self.scene.scene.bounding_box
            self.scene.setup_camera(60, bounds, bounds.get_center())

def main():
    app = QApplication(sys.argv)
    gui.Application.instance.initialize()
    
    window = CalibrationApp()
    window.show()
    
    app.exec()

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'PySide6'